In [1]:
import csv

input_file = 'formated_data.txt'
output_file = 'output.csv'

with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8', newline='') as f_out:
    lines = f_in.readlines()
    header = lines[0].strip().split('\t')

    writer = csv.writer(f_out)
    writer.writerow(header)

    for line in lines[1:]:
        row = line.strip().split('\t')
        writer.writerow(row)

print(f'Данные сохранены в {output_file}')


FileNotFoundError: [Errno 2] No such file or directory: 'formated_data.txt'

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('output.csv')

def find_significant_fields(df, var_threshold=1e-3, missing_threshold=0.1, unique_threshold=1):
    significant_fields = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            missing_ratio = df[col].isna().mean()
            if missing_ratio > missing_threshold:
                continue

            unique_vals = df[col].nunique(dropna=True)
            if unique_vals <= unique_threshold:
                continue

            variance = df[col].var()
            if pd.isna(variance) or variance < var_threshold:
                continue

            significant_fields.append(col)
    return significant_fields

significant_cols = find_significant_fields(df)

df_significant = df[significant_cols]

output_filename = 'cleaned_dataset.csv'
df_significant.to_csv(output_filename, index=False, encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'output.csv'

In [ ]:
df_cleaned = df_significant.drop_duplicates()
df_cleaned = df_cleaned.dropna()

def remove_outliers(df, n_std=3):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        lower_bound = mean - n_std * std
        upper_bound = mean + n_std * std
        df = df.loc[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

df_cleaned = remove_outliers(df_cleaned, n_std=3)

print(f"Размер датасета после очистки: {df_cleaned.shape}")

df_cleaned.to_csv('cleaned_dataset_no_outliers.csv', index=False)
print("Очищенный от выбросов датасет сохранён в cleaned_dataset_no_outliers.csv")


In [3]:
import pandas as pd
import numpy as np

corr_matrix = df_cleaned.corr().abs()

cols_to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        col1 = corr_matrix.columns[i]
        col2 = corr_matrix.columns[j]
        if corr_matrix.iloc[i, j] > 0.98:
            cols_to_drop.add(col2)

print(f"Столбцы, которые будут удалены из-за полной корреляции: {cols_to_drop}")

df_cleaned = df_significant.drop(columns=list(cols_to_drop))

print(f"Размер датафрейма до удаления: {df_significant.shape}")
print(f"Размер датафрейма после удаления: {df_cleaned.shape}")

df_cleaned.to_csv('cleaned_dataset_no_duplicates.csv', index=False)
print("Очищенный датасет сохранён в cleaned_dataset_no_duplicates.csv")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

corr_matrix = df_cleaned.corr()

np.fill_diagonal(corr_matrix.values, 0)

threshold = 0.7

strong_pairs = []
for col in corr_matrix.columns:
    for idx in corr_matrix.index:
        val = corr_matrix.loc[idx, col]
        if pd.notna(val) and abs(val) >= threshold:
            if idx < col:
                strong_pairs.append((idx, col, val))

for x_col, y_col, corr_val in strong_pairs:
    plt.figure(figsize=(6, 4))
    plt.scatter(df_cleaned[x_col], df_cleaned[y_col], alpha=0.5)
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title(f'Диаграмма рассеяния: {x_col} vs {y_col} (corr={corr_val:.2f})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def elbow_method(data, max_clusters=10):
    inertia = []

    for k in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        inertia.append(kmeans.inertia_)

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_clusters + 1), inertia, marker='o')
    plt.title('Метод локтя')
    plt.xlabel('Число кластеров')
    plt.ylabel('Сумма квадратов расстояний (Inertia)')
    plt.xticks(range(1, max_clusters + 1))
    plt.grid()
    plt.show()


def classify_flight_states(data, n_clusters=3):
    features = data[['Vn_ins', 'Ve_ins', 'Vh_ins', 'Om_x_sbkv', 'Om_y_sbkv', 'Om_z_sbkv']]
    kmeans = KMeans(n_clusters=n_clusters)
    data['Flight_State'] = kmeans.fit_predict(features)
    return data

def cluster_statistics(data):
    cluster_stats = data.groupby('Flight_State').describe()
    print("Статистические данные по кластерам:")
    print(cluster_stats)

    return cluster_stats

def cluster_centers(data, n_clusters = 3):
    centers = data.groupby('Flight_State').mean()
    print("Центры кластеров:")
    print(centers)

    return centers

elbow_method(df_cleaned)
data = classify_flight_states(df_cleaned)
cluster_statistics(data)
cluster_centers(data)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def check_normality(data, columns):
    for column in columns:
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        sns.histplot(data[column], kde=True)
        plt.title(f'Гистограмма {column}')

        plt.subplot(1, 2, 2)
        stats.probplot(data[column], dist="norm", plot=plt)
        plt.title(f'Q-Q график {column}')

        plt.tight_layout()
        plt.show()

        stat, p_value = stats.shapiro(data[column])
        print(f'Тест Шапиро-Уилка для {column}: статистика={stat:.3f}, p-значение={p_value:.3f}')

        alpha = 0.05
        if p_value > alpha:
            print(f'{column} подчиняется нормальному распределению (не отвергаем H0)')
        else:
            print(f'{column} не подчиняется нормальному распределению (отвергаем H0)')
        print("\n")

check_normality(data, ['Vn_ins', 'Ve_ins', 'Vh_ins', 'Om_x_sbkv', 'Om_y_sbkv', 'Om_z_sbkv'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def check_uniformity(data, columns):
    for column in columns:
        # Визуализация: гистограмма
        plt.figure(figsize=(8, 4))
        sns.histplot(data[column], bins=20, kde=False)
        plt.title(f'Гистограмма {column}')
        plt.xlabel(column)
        plt.ylabel('Частота')
        plt.grid()
        plt.show()

        data_normalized = (data[column] - data[column].min()) / (data[column].max() - data[column].min())
        stat, p_value = stats.kstest(data_normalized, 'uniform')

        print(f'Тест Колмогорова-Смирнова для {column}: статистика={stat:.3f}, p-значение={p_value:.3f}')

        alpha = 0.05
        if p_value > alpha:
            print(f'{column} подчиняется равномерному распределению (не отвергаем H0)')
        else:
            print(f'{column} не подчиняется равномерному распределению (отвергаем H0)')
        print("\n")

check_uniformity(data, ['Vn_ins', 'Ve_ins', 'Vh_ins', 'Om_x_sbkv', 'Om_y_sbkv', 'Om_z_sbkv'])


In [ ]:
from scipy.stats import skew

def check_symmetry(data, columns):
    for column in columns:
        plt.figure(figsize=(8, 4))
        sns.histplot(data[column], bins=20, kde=True)
        plt.title(f'Гистограмма {column}')
        plt.xlabel(column)
        plt.ylabel('Частота')
        plt.grid()
        plt.show()

        skewness = skew(data[column])
        print(f'Коэффициент асимметрии для {column}: {skewness:.3f}')

        if abs(skewness) < 0.1:
            print(f'{column} имеет симметричное распределение.')
        elif skewness > 0:
            print(f'{column} имеет положительное скошенное распределение.')
        else:
            print(f'{column} имеет отрицательное скошенное распределение.')
        print("\n")

check_symmetry(data, ['Vn_ins', 'Ve_ins', 'Vh_ins', 'Om_x_sbkv', 'Om_y_sbkv', 'Om_z_sbkv'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde

def check_multimodality(data, column):
    data_normalized = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

    plt.figure(figsize=(10, 6))

    sns.histplot(data_normalized, bins=30, kde=False, color='blue', alpha=0.5)

    density = gaussian_kde(data_normalized)
    x = np.linspace(data_normalized.min(), data_normalized.max(), 1000)
    plt.plot(x, density(x), color='red', label='Плотность вероятности')

    plt.title(f'Гистограмма и плотность вероятности для {column}')
    plt.xlabel(column)
    plt.ylabel('Частота')
    plt.legend()
    plt.grid()
    plt.show()


check_multimodality(data, 'Ve_ins')